# Optimization Techniques

## Introduction

In machine learning, optimization is one of most important factor while training a model. Optimization refers to finding the set of parameters that minimize the loss function (or error function). Like in neural networks, we have weights and biases as parameters, in linear regression, we have cofficients. It all boils down to coming up with the best set of learnable parameters that would minimize the loss funtion. We use different optimization techniques in order to iterate and find best set of parameters corresponding to minimum loss (error).

## Gradients

When a given function depends on one variable, we define a derivative as the rate of change of a function with respect to a variable. It tells the slope of a line tangent to function at the point. Similarly, gradient is defined for a function which depends on multiple variables. It is calculated using partial derivatives and stored in Jacobian Matrix. Jacobian Matrix consists of first-order partial derivetives. Gradients can be seen as analogous to derivatives in case of a multivariate function.

## Overview

Broadly speaking otimization techniques can be classified into two categories: First-Order techniques and Second-Order techniques. Frist-Order techniques use gradients (partial derivatives) in order to find the maximum/minimu of a function. Second-Order techniques use second order derivatives which are represented as Hessian. Similar to Jacobian, Hessian is a matrix of second-order partial derivatives. The first order derivative signifies the increase or decrease of a function, second order derviative signifies the increase or decrease of first order derivative. It signifies the curvature of a quadratic tanget to a function at the point.

In general, first order optimization techniques are preferred over second order because they are easier to calculate and converge faster. Most common first order technique used for optimizing error function is gradient descent.

## Gradient Descent

Gradient descent is the most common optimization technique. It tells that in order to find the minimum value of a function, proceed in the direction opposite to its gradient at the point. Moving in the opposite direction of gradient iteratively, leads to a local optima. If one needs to find the maximum value of a function then should proceed in the direction of gradient, which is commonly known as gradient ascent. 

In machine learning, we start with a random set of parameters and predict the output. We define our loss (or error) function and calculate the gradient of loss function with respect to each paramter. We update the parameter value by subtracting the gradient multiplied with a learning rate and keep on iterating this process of updating parameters until it converges.

Parameter Update Rule: $\theta = \theta - \eta \cdot \nabla J(\theta)$

where, $\theta$ is a vector of learnable paramters, $\eta$ is a learning rate, $\nabla J(\theta)$ is a Jacobian with respect to paramters $\theta$.

## Variants of Gradient descent

There are multiple ways to perform gradient descent: 
-  __Batch (Vanilla) Gradient Descent__: In batch gradient descent, we take whole data at a time and perform one update step at a time. Though this is relatively stable and converges but it is computaionally heavy and considerably slow. It might take long to converge.

```python
for i in range(number of epochs):
  params_gradient = evaluate_gradient(loss_function, data, params)
  params = params - learning_rate * params_gradient
```
<br>


- __Stochastic Gradient Descent__: In SGD, we take one training example at a time and perform the parameter update step. Thus, it takes less time than batch gradient descent to converge but it is quite unstable and has high variance.


```python
for i in range(number of epochs):
  np.random.shuffle(data)
  for example in data:
    params_gradient = evaluate_gradient(loss_function, example, params)
    params = params - learning_rate * params_gradient
```
<br>



- __Mini Batch Gradient descent__: Mini batch is most common and it takes good from both batch and SGD. In mini batch, instead of taking whole training set or one example at a time, we take sample of training examples and perform gradient descent, thus, it is stable to converge as well as is relatively faster.



```python
for i in range(number of epochs):
  np.random.shuffle(data)
  for batch in get_batches(data, batch_size=150):
    params_gradient = evaluate_gradient(loss_function, batch, params)
    params = params - learning_rate * params_gradient
```


## Demerits of Gradient descent

Though gradient descent is most commonly used optimization techniques, it has few demerits:
- Correct value of learning rate is always unknown. If we take high value of learning rate then we often overshoot and miss the convergence. Slow learning rate is computaionally slow and evetually takes long time to converge. Often annealing (reducing learning rate with increasing epochs) is used for better results.


- If one is lucky and loss function is convex then gradient descent would lead to the global minima but in real, loss functions (especially in neural networks) are complex and have multiple local optima. Gradient descent often gets stuck in local minima.


- All parameters have same learning rate. This might be an issue if features used in the model have varying frequency. We would like to make larger update for less frequent features.

## Gradient descent optimization

### Momemtum

Momentum helps in overcoming an unstable and high variance nature of SGD. It adds a fraction __$\gamma$__ of previous update step to the current step and has larger update for dimensions whose gradient has the same direction as the previous step and has smaller update for dimensions whose gradient changes its direction. Thus, it prevents irrelevant updates and eventually has reduced oscillations as opposite to SGD, resulting in faster and stable convergance. Usually, $\gamma$ has a value of 0.9 or similar.

Parameter Update Rule:<br>

$V_{(t)} = \gamma V_{(t-1)} + \eta \cdot \nabla J(\theta)$<br>
$\theta = \theta - V_{(t)}$

### Adagrad

Adagrad helps in parameter-wise adapting of learning rate. It makes larger updates for parameters which are less frequent and smaller updates for parameters which are more frequent. For each paramter, at each time step, it uses different learning rate, based on previous gradients. At a given time step, for a given parameter $\theta_i$, we can write it's update rule as:<br>

$g_{t,i} = \nabla_\theta  J(\theta_t, i)$,   where, $g_{t,i}$ is the gradient of loss function w.r.t. to a parameter $\theta_i$ at a time step t<br>
$\theta_{t+1,i} = \theta_{t,i} - \eta \cdot g_{t,i}$<br>

Modifying the learning rate $\eta$ in the above update rule for a given parameter $\theta_i$:<br>

$\theta_{t+1,i} = \theta_{t,i} - \frac{\eta}{\sqrt{G_{t,ii} + \epsilon}} \cdot g_{t,i}$ <br>
where, $G_t$ is the diagonal matrix in which each diagonal element (or $ii^{th}$ element of matrix) is the sum of squares of previous gradients of loss function w.r.t. $\theta_i$ upto time step t and $\epsilon$ is a smoothening factor to avoid divison by 0.

The above rule is for a single parameter $\theta_i$. This can be written in the vectorized form as follows: <br>

$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{G_t + \epsilon}}\odot \ g_t$

The major drawback of Adagrad is the accumulation of sum squared gradients in the denominator. During the course of training, the sum keeps on increasing making the denominator larger and larger, subsequently, making the learning rate too small for the network to learn anything further. This issue is addressed in Adadelta.

### Adadelta

Adadelta is an extension to Adagrad that address the decaying learning rate issue. It has two major modifications over Adagrad. First, instead of accumulating all previous gradients, here, we restrict the accumulated previous gradient to a fixed size of w. Second, It does not directly adds w previous square gradients as was in the case of Adagrad but instead it efficiently accumulates the decaying average of previous squared gradients. Thus, the current average depends only on a fraction of average previous squared gradients and current gradient.

$E[g^2]_t = \gamma E[g^2]_{t−1} + (1−\gamma)g^2_t$ <br>

where, $\gamma$ is similar to momentum and usually set to 0.9 <br>

Replacing $G_t$ term with $E[g^2]_t$ in Adagrad update rule, we get: <br>

$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{E[g^2]_t + \epsilon}}\odot \ g_t$


### Adam

In addition to storing exponentially decaying average of past squared gradients $v_t$ like Adadelta, Adam stores exponentially decaying average of previous gradients $m_t$, as well, similar to momentum.

$v_t$ and $m_t$ are the first moment (mean) and second moment (uncentered variance) of gradients, respectively.<br>

$\hat{m_t} = \frac{m_t}{1-\beta_1^t}$<br>
$\hat{v_t} = \frac{v_t}{1-\beta_2^t}$<br>

The parameter update rule becomes: <br>

$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{\hat{v_t}} + \epsilon} \hat{m_t}$<br>

The default values of 0.9 and 0.999 are used for $\beta_1$ and $\beta_2$, respectively.

In practice, Adam performs well in comparison to others. It also overcomes different issues encountered in other techniques, such as, high vairance, slow convergence, diminishing learning rate, etc.

In general, adaptive learning rate techniques, such as, Adagrad, Adadelta, Adam, etc. perform better than SGD, momentum, etc. and converge relatively faster. One additional advantage of adaptive leraning rate techniques is that we don't need to manually tune learning rate. They pretty much converge well with default value.